# Deep Learning - Assignment 1

#### Submitted By: Kalyani Prashant Kawale
#### Student ID: 21237189

## Solution:

In [2]:
# Package imports
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import pandas as pd
import numpy as np
import itertools
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Display plots inline and change default figure size
%matplotlib inline

### Task 1: Implement Logistic Regression

In [ ]:
def LogisticRegressor(X, y, alpha, max_iters):
    (nsamples, nattributes) = np.shape(X)
    threshold = 1e-6
    w = np.random.rand(nattributes)
    b = np.random.rand()
    J_prev = 0
    
    for i in range(max_iters):   
        idx = np.random.choice(nsamples, 1, replace=False)
        random_X = X[idx[0]]
        random_y = y[idx[0]]
        y_hat = 1 / (1 + np.exp(-1 * (np.dot(w, random_X) + b)))
        J_curr = -1 * ((random_y * np.log(y_hat)) + ((1 - random_y) * np.log(1 - y_hat)))
        if np.absolute(J_curr - J_prev) < threshold:
            break
        else:
            J_prev = J_curr        
        delta_w = []
        for j in range(len(w)):
            delta_w.append((y_hat - random_y) * random_X[j])
        delta_b = (y_hat - random_y)
        for j in range(len(w)):
            w[j] -= alpha * delta_w[j]
        b -= alpha * delta_b
        
    return w, b

def predict(data, weights, bias):
    predictions = []
    for sample in data:
        prediction = 1 / (1 + np.exp(-1 * (np.dot(weights, sample) + bias))) 
        if prediction >= 0.5:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions

### Task 2: Training, Validating, Testing on blobs300 and circles600 Datasets

In [ ]:
files = ["blobs300.csv", "circles600.csv"]
alpha_s = [0.1, 0.001, 0.0001]
iterations = [1000, 10000, 20000]
best_alpha = 0.1
best_iterations = 1000
best_accuracy = 0
X_train = {}
y_train = {}
X_valid = {}
y_valid = {}
X_test = {}
y_test = {}
results = pd.DataFrame(columns=['File Name', 'Learning Rate', 'Iterations', 'Accuracy'])
np.random.seed(200)
for file in files:
    # Use pandas to read the CSV file as a dataframe
    df = pd.read_csv(file).sample(frac=1).reset_index(drop=True)
    
    # The y values are those labelled 'Class': extract their values
    y = df['Class'].values

    # The x values are all other columns
    del df['Class']   # drop the 'Class' column from the dataframe
    X = df.values     # convert the remaining columns to a numpy array
    
    X_train[file], X_validate_test, y_train[file], y_validate_test = train_test_split(X, y, train_size=0.7)

    X_valid[file], X_test[file], y_valid[file], y_test[file] = train_test_split(X_validate_test, y_validate_test, test_size=0.5)
    
    for alpha, iters in itertools.product(alpha_s, iterations):
        row = {'File Name': file, 'Learning Rate': alpha, 'Iterations': iters}
        w, b = LogisticRegressor(X_train[file], y_train[file], alpha, iters)
        predictions = predict(X_valid[file], w, b)
        accuracy = accuracy_score(y_valid[file], predictions)
        row['Accuracy'] = accuracy
        results = results.append(row, ignore_index=True)
        if accuracy > best_accuracy:
            best_alpha = alpha
            best_iterations = iters
            best_accuracy = accuracy


In [ ]:
print("Training and Validation Results:\n")
print(results.to_markdown())

In [ ]:
np.random.seed(200)
print("Testing Results:")
for file in files:
    print(file)
    w, b = LogisticRegressor(X_train[file], y_train[file], best_alpha, best_iterations)
    predictions = predict(X_test[file], w, b)
    print(f"Accuracy: {accuracy_score(y_test[file], predictions)}")

### Task 3: Shallow Neural Network:

In [91]:
def f(z):
    return 1 / (1 + np.exp(-1 * z))

def f_dash(z):
    return f(z) * (1 - f(z))

def ForwardPropagation(x, weights, biases):
    sigmas = []
    activations = []
    # Initialising activations with input values for layer 1
    activations.append(x)    
    for layer in range(len(weights)):
#         print(f"Layer {layer}")
#         print(weights[layer].shape)
        sigma_layer = np.zeros(weights[layer].shape[0])
        activations_layer = np.zeros(weights[layer].shape[0])
        for i in range(weights[layer].shape[0]):            
            sigma = 0
            for j in range(weights[layer].shape[1]):
                sigma += weights[layer][i][j] * activations[layer][j]
            sigma_layer[i] = sigma + biases[layer][i]
            activations_layer[i] = f(sigma_layer[i])
        sigmas.append(sigma_layer)
        activations.append(activations_layer)
            
    return sigmas, activations

def BackPropagation(y, activations, sigmas, weights):
    delta_weights = []
    delta_biases = []
    delta_sigmas = [activations[len(activations)-1] - y]
#     print(f"delta_sigmas {delta_sigmas}")
    for i in range(weights[len(weights) - 1].shape[0]):
        delta_weight = []
        for j in range(weights[len(weights) - 1].shape[1]):
            delta_weight.append(delta_sigmas[i][0] * activations[len(activations)-2][j])
        delta_weights.append(np.array([delta_weight]))
        
    for layer in range(len(weights) - 1):
#         print(f"at layer {layer}")
        # first finding the delta sigmas
        delta_sigma_layer = np.zeros(sigmas[layer].shape[0])
        for i in range(sigmas[layer].shape[0]):
#             print(f"sigma at layer {layer} and {i}th position {sigmas[layer][i]}")
            delta_sigma = f_dash(sigmas[layer][i])            
            summation = 0            
            for j in range(weights[layer + 1].shape[0]):
                for k in range(weights[layer + 1].shape[1]):
                    summation += weights[layer + 1][j][k] * delta_sigmas[layer - 1][j]      
            
            delta_sigma_layer[i] = delta_sigma * summation
        delta_sigmas.append(delta_sigma_layer)
        
        delta_layer_weights = np.zeros(weights[layer].shape)
        for i in range(weights[layer].shape[0]):
            for j in range(weights[layer].shape[1]):
                delta_layer_weights[i][j] = delta_sigmas[layer + 1][i] * activations[layer][j]
        delta_weights.append(delta_layer_weights)
    
    delta_biases = delta_sigmas
                
    return list(reversed(delta_weights)), list(reversed(delta_biases))

def SGD_Parameter_Update(alpha, weights, biases, delta_weights, delta_biases):
    
    for layer in range(len(weights)):
        for i in range(weights[layer].shape[0]):    
            for j in range(weights[layer].shape[1]):
                weights[layer][i][j] -= alpha * delta_weights[layer][i][j]
    
    for layer in range(len(biases)):
        for i in range(biases[layer].shape[0]):
            biases[layer][i] -= alpha * delta_biases[layer][i]
    
    return weights, biases
                
def predictNN(data, weights, biases):
    predictions = []
    for sample in data:
        sigmas, activations = ForwardPropagation(sample, weights, biases)
        prediction = activations[len(activations) - 1][0]
        if prediction > 0.5:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions
    
def LogisticRegressor_1NN(X, y, alpha, max_iters, layers=2, hidden_layer_neurons=3):
    errors = []
    loss = []
    (nsamples, nattributes) = np.shape(X)
    threshold = 1e-6
    
    layer_neurons = []
    for layer in range(layers-1):
        layer_neurons.append(hidden_layer_neurons)
    layer_neurons.append(1)
    layer_neurons.insert(0, nattributes)
    
    weights = []
    biases = []
    for i in range(1, layers + 1):
        weights.append(np.random.rand(layer_neurons[i], layer_neurons[i - 1]))
        biases.append(np.array(np.random.rand(layer_neurons[i])))
    print("weights")
    print(weights)
    print("Biases")
    print(biases)
    
    J_prev = 0
        
    for iters in range(max_iters):
        idx = np.random.choice(nsamples, 1, replace=False)
        random_X = X[idx[0]]
        random_y = y[idx[0]]
        sigmas, activations = ForwardPropagation(random_X, weights, biases)

        y_hat = activations[len(activations)-1][0]        
        J_curr = -1 * ((random_y * np.log(y_hat)) + ((1 - random_y) * np.log(1 - y_hat)))        
        delta_weights, delta_biases = BackPropagation(random_y, activations, sigmas, weights)
        if np.absolute(J_curr - J_prev) < threshold:
            break
        else:
            J_prev = J_curr
        
        weights, biases = SGD_Parameter_Update(alpha, weights, biases, delta_weights, delta_biases)
            
    return weights, biases, errors, loss
                

In [146]:
df = pd.read_csv("blobs300.csv")
# df = pd.read_csv("circles600.csv")
y = df['Class'].values
del df['Class']
X = df.values   
# np.random.seed(200)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7)
(nsamples, nattributes) = X_train.shape
#300
# weights, biases, errors, loss = LogisticRegressor_1NN(X_train, y_train, 0.1, 2000, layers=2, hidden_layer_neurons=int(nattributes / 2))
# 600
weights, biases, errors, loss = LogisticRegressor_1NN(X_train, y_train, 0.1, 10000, layers=2, hidden_layer_neurons=4)
# print("UPDATED WEIGHTS AND BIASES")
# print(weights)
# print(biases)

predictions = predictNN(X_valid, weights, biases)
print(f"Accuracy: {accuracy_score(y_valid, predictions)}")

weights
[array([[0.78863355, 0.84340158, 0.07328244, 0.75937946],
       [0.04413816, 0.10103551, 0.97446269, 0.51075187],
       [0.84742544, 0.93746915, 0.62025996, 0.3543248 ],
       [0.43755636, 0.40304762, 0.99846879, 0.83245862]]), array([[0.28449671, 0.12503889, 0.9858171 , 0.84251261]])]
Biases
[array([0.59418336, 0.3475767 , 0.49891584, 0.65814956]), array([0.77493989])]
Accuracy: 1.0
